<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Keras_WithDeeplizard/blob/master/25_Learnable_Parameters_(%22Trainable_Params%22)_In_A_Keras_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trainable Parameters In A Keras Convolutional Neural Network

在本节课中，我们将讨论如何使用Keras代码快速访问和计算卷积神经网络中可学习参数的数量。我们还将探索这些参数可能如何受到其他可选配置的影响

## 1.带有零填充的Keras模型
我们有一个非常基本的Keras卷积神经网络

In [ ]:
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.layers.convolutional import *
from keras.layers.pooling import *

model = Sequential([
    Conv2D(
        2
        , kernel_size=(3,3)
        , input_shape=(20,20,3)
        , activation='relu'
        , padding='same'
    ),
    Conv2D(
        3
        , kernel_size=(3,3)
        , activation='relu'
        , padding='same'
    ),
    Flatten(),
    Dense(
        2, 
        activation='softmax'
    )
])

我们可以在卷积层中看到，我们指定的填充与我们的填充相同，我们从另一集中知道填充是零填充。

在以前的Keras章节中，我们还看到了如何通过调用模型上的summary()函数并检查Param #列来查看Keras模型每一层中可学习参数的数量，以及整个网络中的参数数量。





In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 20, 20, 2)         56        
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 3)         57        
_________________________________________________________________
flatten_1 (Flatten)          (None, 1200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2402      
Total params: 2,515
Trainable params: 2,515
Non-trainable params: 0
_________________________________________________________________


我们的模型有这个总结输出，实际上这个模型是我们在[深度学习基础系列](https://deeplizard.com/learn/playlist/PLZbbT5o_s2xq7LwI2y8_QtvuXZedL6tQU)中学习[如何计算CNN中可学习参数数量](https://deeplizard.com/learn/video/gmBfb6LNnZs)时使用的概念模型的精确实现。

如果你还记得那节课，在我们的第一卷积层中，我们确实计算出了56个可学习参数，就像Keras在这个输出中向我们展示的那样。我们还计算出第二层卷积层包含57个可学习参数，输出层包含2402个参数，使我们在整个网络中总共有2515个可学习参数。

现在，记住，我们在这里使用零填充来保持图像在网络中流动时的尺寸。我们之前已经看到，当我们在计算输出密集层中可学习参数的数量时，维度起了作用。

我们需要计算有多少输入进入这个层，我们计算为1200，如Flatten层的输出形状列所示。通过乘以20x20x3得到1200这个数字，其中3是最后一个卷积层的滤波器数量。

20x20来自图像数据的维数，因为它是由前一个卷积层输出的。我们可以把这些尺寸看成是第二个卷积层的输出形状。然后我们用1200乘以输出层的2个节点，并加上2个偏置项，得到2402的结果。

如果你没有完全理解我刚才总结的计算方法，那就用我之前提到的计算CNNs中可学习参数数量的章节来刷新你的记忆。

## 2.不带零填充的Keras模型
现在，如果我们不使用零填充，那么这会对模型中可学习参数的数量产生什么影响呢?我们去看看。

In [ ]:
model = Sequential([
    Conv2D(2, kernel_size=(3,3), input_shape=(20,20,3), activation='relu'),
    Conv2D(3, kernel_size=(3,3), activation='relu'),
    Flatten(),
    Dense(2, activation='softmax')
])

这与我们刚刚使用的模型是完全相同的，除了现在我们不再使用零填充，所以我们不再在两个卷积层中指定填充参数。

In [ ]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 18, 18, 2)         56        
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 16, 16, 3)         57        
_________________________________________________________________
flatten_2 (Flatten)          (None, 768)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 1538      
Total params: 1,651
Trainable params: 1,651
Non-trainable params: 0
_________________________________________________________________


两个卷积层的可学习参数数量保持不变，但我们可以看到最后一个密集层的可学习参数数量从2402大幅下降到1538。

那是因为，图像的尺寸在离开最后的卷积层时已经缩小到了16x16，所以现在，不是用20x20x3乘以1200，而是用16x16x3，得到768。

所以，仅仅通过去除卷积层的零填充，网络中总的可学习参数数量就从2515下降到1651，下降了34%。

## 3.带有零填充和最大池层的Keras模型
现在，让我们把零填充放回我们的模型中，让我们看看如果我们在我们的模型中添加一个最大池层，对可学习参数的数量会有什么影响。毕竟，在CNN中使用最大池是非常传统的。


In [ ]:
model = Sequential([
    Conv2D(2, kernel_size=(3,3), input_shape=(20,20,3), activation='relu', padding='same'),
    Conv2D(3, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2,2),strides=2),
    Flatten(),
    Dense(2, activation='softmax')
])

这是我们的原始模型，使用相同的架构，使用零填充，但是现在，我们在第二个卷积层之后添加了一个最大池化层。我们指定的池大小是2x2，步长为2。

我们从之前的max pooling中了解到这将会减少图像的尺寸。实际上，pool_size和stride的这种特定选择将维度减少了一半。我们可以在这个最大池化层的输出形状列中看到这一点。

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 20, 20, 2)         56        
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 20, 20, 3)         57        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 3)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 602       
Total params: 715
Trainable params: 715
Non-trainable params: 0
_________________________________________________________________


所以现在，当我们平展卷积输出时，我们不是乘以原来的20x20x3维数，而是乘以10x10x3，作为最大池化的结果。这大大减少了输出层中的可学习参数，从最初的2402减少到602，这使得网络中的可学习参数总数从2515减少到715。


这就是我们如何在Keras中访问和确认CNN中可学习参数的总数，以及看看这些常见的零填充和最大池化技术对我们模型中可学习参数的数量有什么样的影响。下期见

